#Import Data

In [2]:
!git clone -l -s git://github.com/AshraqatM/NLP_clustering.git cloned-repo
%cd cloned-repo
!ls

Cloning into 'cloned-repo'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 3 (delta 0), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (3/3), 1.44 MiB | 3.41 MiB/s, done.
/content/cloned-repo
oversampling_dataset.csv


In [20]:
import pandas as pd
import numpy as np
df = pd.read_csv("oversampling_dataset.csv")
df.head()
df['category'] = df['category'].replace(['none','damning', 'derailing', 'discredit', 'dominance' , 'sexual harassment' , 'stereotyping & objectification' , 'threat of violence'],[0,1,2,3,4,5,6,7]) 
df = df[['text','category']]


#Install Simple Transformers library

In [4]:
# install simpletransformers
!pip install simpletransformers

# check installed version
!pip freeze | grep simpletransformers
# simpletransformers==0.28.2

     |████████████████████████████████| 221 kB 8.9 MB/s 
     |████████████████████████████████| 1.2 MB 48.3 MB/s 
     |████████████████████████████████| 2.5 MB 45.3 MB/s 
     |████████████████████████████████| 6.6 MB 60.1 MB/s 
     |████████████████████████████████| 43 kB 2.1 MB/s 
     |████████████████████████████████| 76 kB 5.4 MB/s 
     |████████████████████████████████| 3.3 MB 58.4 MB/s 
     |████████████████████████████████| 1.8 MB 48.8 MB/s 
     |████████████████████████████████| 124 kB 66.4 MB/s 
     |████████████████████████████████| 262 kB 69.6 MB/s 
     |████████████████████████████████| 895 kB 56.7 MB/s 
     |████████████████████████████████| 97 kB 8.8 MB/s 
     |████████████████████████████████| 138 kB 71.8 MB/s 
     |████████████████████████████████| 170 kB 73.7 MB/s 
     |████████████████████████████████| 133 kB 60.1 MB/s 
     |████████████████████████████████| 63 kB 2.2 MB/s 
     |████████████████████████████████| 62 kB 1.0 MB/s 
     |███████████████████

simpletransformers==0.61.10


#Spliting

In [21]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(df, test_size=0.30)

print('train shape: ',train_df.shape)
print('test shape: ',test_df.shape)
train_df

train shape:  (14862, 2)
test shape:  (6370, 2)


,text,category
20971,اكبر الاع الاعوج الي انتي باجرينا باشوكي بركي ...,7
6037,البلاء الكربلاء بحق بصوت تزقك تعملك حبه دعوات ...,1
18602,الراء ام حرف لسان نص ويبتقرط,6
6159,اله ديمالوطيه صباحك لا ولا ومساكي يخليكي يسعد ...,1
14985,اساس الفتنه انت بالتفلون بوزك تنشريلنا رقمك ري...,5
...,...,...
14139,انزل زعره شوفك عبالي فوق كتير لان لتحت مشتقلك ...,5
8156,الا البلد البهدله بتحبي تسمعي جبران ديما سقوط ...,2
11451,ات اشرف اكثر العالم الي بكرامه تعيشي خلينكي صب...,4
1746,اعلاميه المعلومات اين بتنشري بصفتك حضرتك شو لك...,0


#Pre-training

In [6]:
# 2-layer, 768-hidden, 12-heads, 179M parameters. Trained on cased text in the top 104 languages with the largest Wikipedias
from simpletransformers.classification import ClassificationModel

# define hyperparameter
train_args ={"reprocess_input_data": True,
             "overwrite_output_dir": True,
             "fp16":False,
             "num_train_epochs": 4}

# Create a ClassificationModel
model = ClassificationModel(
    "bert", "bert-base-multilingual-cased",
    num_labels=8,
    args=train_args
)

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

Downloading:   0%|          | 0.00/996k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

#Train our Model

In [22]:
# Train the model
model.train_model(train_df)

/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:616: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


  0%|          | 0/14862 [00:00<?, ?it/s]

Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

Running Epoch 0 of 4:   0%|          | 0/1858 [00:00<?, ?it/s]

Running Epoch 1 of 4:   0%|          | 0/1858 [00:00<?, ?it/s]

Running Epoch 2 of 4:   0%|          | 0/1858 [00:00<?, ?it/s]

Running Epoch 3 of 4:   0%|          | 0/1858 [00:00<?, ?it/s]

(7432, 0.24919135948761395)

In [23]:
from sklearn.metrics import f1_score, accuracy_score


def f1_multiclass(labels, preds):
    return f1_score(labels, preds, average='micro')
    
result, model_outputs, wrong_predictions = model.eval_model(test_df, f1=f1_multiclass, acc=accuracy_score)

result

/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:1402: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


  0%|          | 0/6370 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/797 [00:00<?, ?it/s]

{'acc': 0.9657770800627944,
 'eval_loss': 0.1970445370585832,
 'f1': 0.9657770800627944,
 'mcc': 0.9608917922605458}

array([[-0.70211124, -1.17822301, -1.45909417, ..., -1.58958888,
        -1.67530477,  9.63980675],
       [ 8.13375282, -1.65899348, -1.96035528, ..., -2.25156927,
        -1.46093631, -1.90875685],
       [-1.47079515, -1.37758315, -1.64475298, ..., -1.35730755,
        -0.76126516, -1.38488698],
       ...,
       [-0.4915455 , -0.32965928, -2.59261513, ..., -2.49169827,
         8.694561  , -2.73320127],
       [-1.39389956, -1.15259504, -0.94959384, ..., 11.12267113,
        -1.10498822, -0.84060037],
       [-1.39355588, -1.10233235, -0.93322456, ..., 11.13212204,
        -1.1578325 , -0.74132925]])

#save the model

In [29]:
from google.colab import drive
drive.mount('drive')

Mounted at drive


In [39]:
import os
import tarfile

def save_model(model_path='',file_name=''):
  files = [files for root, dirs, files in os.walk(model_path)][0]
  with tarfile.open(file_name+ '.tar.gz', 'w:gz') as f:
    for file in files:
      f.add(f'{model_path}/{file}')

In [40]:
save_model('outputs','mber_text')

In [41]:
!tar -zxvf ./mber_text.tar.gz


outputs/pytorch_model.bin
outputs/tokenizer.json
outputs/model_args.json
outputs/training_args.bin
outputs/config.json
outputs/tokenizer_config.json
outputs/vocab.txt
outputs/eval_results.txt
outputs/special_tokens_map.json
